<a href="https://colab.research.google.com/github/Mix1996/Project-2-Classification/blob/main/Neural_Networks_(Core).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.impute import SimpleImputer


from sklearn.preprocessing import StandardScaler, OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from sklearn.metrics import mean_squared_error

###Load the Data

In [ ]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTDNQngOuRqfhB2f_hcm1bW456aRw2hwoLFRaVvJ0EVB1l-NNqDe8Ya4hpV0qBf5cPkhUwldXQoIWXI/pub?output=csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


##Explore the data

In [ ]:
df.duplicated().sum()

0

In [ ]:
df.isna().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [ ]:
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [ ]:
# Check for missing values in the entire DataFrame
missing_values = df.isnull().sum()

# Display missing values count for each column
print(missing_values)

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64


In [ ]:
# Assuming 'column_name' is the name of the column you want to drop
df = df.drop('Id', axis=1)


In [ ]:
# Display the sum of missing values
df.isna().sum().sum()

6965

#Modeling

In [ ]:
# Create an imputer for numeric columns
numeric_imputer = SimpleImputer(strategy='mean')

# Fit and transform the imputer on numeric columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = numeric_imputer.fit_transform(df[numeric_cols])


In [ ]:
# Create an imputer for categorical columns
categorical_imputer = SimpleImputer(strategy='most_frequent')

# Fit and transform the imputer on categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])


In [ ]:
# Split the data into features (X) and target variable (y)
X = df.drop(columns=['SalePrice'])  # Features
y = df['SalePrice']  # Target variable

# Split categorical and numerical columns
categorical_cols = [col for col in X.columns if X[col].dtype == 'object']
numerical_cols = [col for col in X.columns if col not in categorical_cols]



In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Data Preprocessing

In [ ]:
# Data Preprocessing
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[numerical_cols])
X_test_num = scaler.transform(X_test[numerical_cols])

In [ ]:
# One-hot encode categorical variables
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_train_cat = encoder.fit_transform(X_train[categorical_cols])
X_test_cat = encoder.transform(X_test[categorical_cols])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Combine numerical and one-hot encoded categorical data
X_train = np.hstack((X_train_num, X_train_cat))
X_test = np.hstack((X_test_num, X_test_cat))

##Define and Train Models

#### Iteration 1: Basic Neural Network

In [ ]:
# Iteration 1: Basic Neural Network
model1 = Sequential()
model1.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
model1.add(Dense(64, activation='relu'))
model1.add(Dense(1, activation='linear'))
model1.compile(loss='mean_squared_error', optimizer='adam')
model1.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
30/30 [==============================] - 1s 7ms/step - loss: 39143211008.0000 - val_loss: 37830635520.0000
Epoch 2/50
30/30 [==============================] - 0s 4ms/step - loss: 39114219520.0000 - val_loss: 37773250560.0000
Epoch 3/50
30/30 [==============================] - 0s 4ms/step - loss: 39009415168.0000 - val_loss: 37605347328.0000
Epoch 4/50
30/30 [==============================] - 0s 4ms/step - loss: 38757060608.0000 - val_loss: 37251694592.0000
Epoch 5/50
30/30 [==============================] - 0s 4ms/step - loss: 38271295488.0000 - val_loss: 36621983744.0000
Epoch 6/50
30/30 [==============================] - 0s 4ms/step - loss: 37478371328.0000 - val_loss: 35646676992.0000
Epoch 7/50
30/30 [==============================] - 0s 6ms/step - loss: 36297723904.0000 - val_loss: 34271102976.0000
Epoch 8/50
30/30 [==============================] - 0s 5ms/step - loss: 34695917568.0000 - val_loss: 32456495104.0000
Epoch 9/50
30/30 [==============================] - 0s 5

### Iteration 2: Neural Network with Regularization (Dropout)

In [ ]:
# Iteration 2: Neural Network with Regularization (Dropout)
model2 = Sequential()
model2.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
model2.add(Dropout(0.5))  # Dropout layer for regularization
model2.add(Dense(64, activation='relu'))
model2.add(Dense(1, activation='linear'))
model2.compile(loss='mean_squared_error', optimizer='adam')
model2.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
30/30 [==============================] - 1s 8ms/step - loss: 39144460288.0000 - val_loss: 37834498048.0000
Epoch 2/50
30/30 [==============================] - 0s 4ms/step - loss: 39124619264.0000 - val_loss: 37792948224.0000
Epoch 3/50
30/30 [==============================] - 0s 4ms/step - loss: 39045554176.0000 - val_loss: 37659934720.0000
Epoch 4/50
30/30 [==============================] - 0s 4ms/step - loss: 38835519488.0000 - val_loss: 37355692032.0000
Epoch 5/50
30/30 [==============================] - 0s 4ms/step - loss: 38409834496.0000 - val_loss: 36792811520.0000
Epoch 6/50
30/30 [==============================] - 0s 4ms/step - loss: 37694398464.0000 - val_loss: 35886338048.0000
Epoch 7/50
30/30 [==============================] - 0s 4ms/step - loss: 36565594112.0000 - val_loss: 34555088896.0000
Epoch 8/50
30/30 [==============================] - 0s 4ms/step - loss: 35012587520.0000 - val_loss: 32774774784.0000
Epoch 9/50
30/30 [==============================] - 0s 4

### Define and train your CNN model

In [ ]:
# Define and train your CNN model
model_cnn = Sequential()
model_cnn.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(128, activation='relu'))
model_cnn.add(Dense(64, activation='relu'))
model_cnn.add(Dense(1, activation='linear'))
model_cnn.compile(loss='mean_squared_error', optimizer='adam')
model_cnn.fit(X_train[:, :, np.newaxis], y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
30/30 [==============================] - 2s 32ms/step - loss: 39117508608.0000 - val_loss: 37730770944.0000
Epoch 2/50
30/30 [==============================] - 1s 25ms/step - loss: 38727602176.0000 - val_loss: 36783730688.0000
Epoch 3/50
30/30 [==============================] - 1s 24ms/step - loss: 36539514880.0000 - val_loss: 32647729152.0000
Epoch 4/50
30/30 [==============================] - 1s 23ms/step - loss: 29480790016.0000 - val_loss: 21975480320.0000
Epoch 5/50
30/30 [==============================] - 1s 24ms/step - loss: 16287461376.0000 - val_loss: 8024788480.0000
Epoch 6/50
30/30 [==============================] - 1s 25ms/step - loss: 6336023040.0000 - val_loss: 4265261312.0000
Epoch 7/50
30/30 [==============================] - 1s 23ms/step - loss: 5240679936.0000 - val_loss: 4078445056.0000
Epoch 8/50
30/30 [==============================] - 1s 24ms/step - loss: 5018542080.0000 - val_loss: 3924458240.0000
Epoch 9/50
30/30 [==============================] - 1s 

#Evaluation

In [ ]:
# Evaluate each model
# Define the evaluate_model function
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test[:, :, np.newaxis])
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    return rmse
rmse1 = evaluate_model(model1, X_test, y_test)
rmse2 = evaluate_model(model2, X_test, y_test)
rmse_cnn = evaluate_model(model_cnn, X_test, y_test)

print("RMSE for Model 1 (Basic NN):", rmse1)
print("RMSE for Model 2 (NN with Dropout):", rmse2)
print("RMSE for CNN Model:", rmse_cnn)

10/10 [==============================] - 0s 6ms/step
RMSE for Model 1 (Basic NN): 39014.979395561284
RMSE for Model 2 (NN with Dropout): 40486.67458672934
RMSE for CNN Model: 37083.083302118765


#Best Model

A lower RMSE indicates a better-performing model in terms of prediction accuracy. Therefore, in this case, the CNN Model with an RMSE of 37,083.08 is the best-performing model among the three iterations. It has the lowest prediction error on your test data.

I will select the CNN Model as the best model for predicting housing prices based on the evaluation results.